The following code defines hyperparameter tuning for LSTM model designed for multi-modal dataset. 

In [ ]:
import pandas as pd
import os
import numpy as np
import json

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras_tuner import Hyperband
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X

In [ ]:
scaler = StandardScaler()

## Data preparation

### X data - audio and visual features

In [ ]:
x_train_visual = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)
x_train_audio = prepare_x_data('../Data/LLDs_audio_eGeMAPS/train',';', 2, scaler) 

### Y data - YMRS score

In [ ]:
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train_visual),1,1)

## Hyperparameter Tuner setup

In [ ]:
def model_builder(hp):
    
    # Hyperparameters for tuning
    hp_units_visual_1 = hp.Int('units_visual_1', min_value=1, max_value=465, step=1)
    hp_units_visual_2 = hp.Int('units_visual_2', min_value=1, max_value=465, step=1)

    hp_units_audio_1 = hp.Int('units_audio_1', min_value=1, max_value=23, step=1)
    hp_units_audio_2 = hp.Int('units_audio_2', min_value=1, max_value=23, step=1)

    hp_droput_visual = hp.Choice('dropout_visual', values=[1e-2, 1e-3, 1e-4])
    hp_droput_audio = hp.Choice('dropout_audio', values=[1e-2, 1e-3, 1e-4])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #learning rate for the optimizer
    
    """
    Multi modal LSTM setup
    """
    # Defining model layers
    visual_input = keras.Input(shape=(None,465), name="visual")
    audio_input = keras.Input(shape=(None,23), name="audio")
    
    visual_features = layers.LSTM(units = hp_units_visual_1, input_shape=(None, 465), return_sequences=True)(visual_input)
    visual_features = layers.Dropout(hp_droput_visual)(visual_features)
    visual_features = layers.LSTM(units = hp_units_visual_2, input_shape=(None, hp_units_visual_1), return_sequences=False)(visual_features)
    visual_features = layers.Dropout(hp_droput_visual)(visual_features)

    audio_features = layers.LSTM(units = hp_units_audio_1, input_shape=(None,23), return_sequences=True)(audio_input)
    audio_features = layers.Dropout(hp_droput_audio)(audio_features)
    audio_features = layers.LSTM(units = hp_units_audio_2, input_shape=(None,hp_units_audio_1), return_sequences=False)(audio_features)
    visual_features = layers.Dropout(hp_droput_audio)(visual_features)

    x = layers.concatenate([visual_features, audio_features])
    x = layers.Dense(1, activation='linear')(x)
    
    # Defining model output
    y_pred =layers.Dense(1, name="ymrs")(x)
    
    # Initializing the model
    model = keras.Model(
    inputs=[visual_input, audio_input],
    outputs=[y_pred]
    )

    # Compiling the model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='mse',           
        metrics=['mean_absolute_error']
    )
    
    return model

In [ ]:
#Initilize tuners

tuner = Hyperband(
    model_builder,
    objective = 'loss',
    max_epochs = 50, 
    factor = 3, 
    project_name = 'Hp_tuner_multi'
)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
index = 0
for train_visual, train_audio in zip(x_train_visual, x_train_audio):
    train_visual = reshape_X(train_visual)
    train_audio = reshape_X(train_audio)

    tuner.search([train_visual, train_audio], y_train[index], epochs=10, callbacks=[stop_early]) 
    index += 1

### Finding the final hyperparameter values

The approach of finding the final hyperparameters for the model is based on extracting the average value from all tuner search iterations. 

In [ ]:
# Combining learned hyperparameters from every iteration in one dataset

def get_hp_df(tunerdir):
    rootdir = tunerdir
    hp_values = []
    #Get learned hyperparameters for every file (trial) and append to a list
    for trial in os.listdir(rootdir):
        pathdir = os.path.join(rootdir, trial)
        filedir = os.path.join(pathdir, 'trial.json')
        
        if os.path.isdir(pathdir): #looking only for trial subfolders
            with open(filedir) as json_file:
                data = json.load(json_file)
                values = data['hyperparameters']['values'] #get learned hyperparameters
                hp_values.append(values) #append to list
        
    #Transform to dataframe
    hp_df = pd.DataFrame(hp_values)

    return hp_df

In [ ]:
#Get hyperparameter values for both modalities
hpdf_multi = get_hp_df('../LSTM/Hp_tuner_multi')

In [ ]:
print("Hyperparameters for the multi-modal LSTM setup:")
hpdf_multi.mean()